In [1]:
from obspy import read
from obspy import UTCDateTime
import matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path
import os

In [2]:
# Load the pick data
pick_data = pd.read_csv("dataset_earthquakes/metadata.csv", usecols=['trace_name_original_Z', 'trace_p_pick_time', 'trace_s_pick_time'])
pick_data


,trace_name_original_Z,trace_p_pick_time,trace_s_pick_time
0,34692381_2023-04-15T13:17:55.541619Z_WS.POZA.S...,2023-04-15T13:18:17.645000Z,2023-04-15T13:18:19.215000Z
1,34692381_2023-04-15T13:17:55.542715Z_WS.POZA.S...,2023-04-15T13:18:17.654000Z,2023-04-15T13:18:19.233000Z
2,34692381_2023-04-15T13:17:55.540100Z_WS.POZA.S...,2023-04-15T13:18:17.642000Z,2023-04-15T13:18:19.253000Z
3,34692381_2023-04-15T13:17:55.539436Z_WS.POZA.S...,2023-04-15T13:18:17.634000Z,2023-04-15T13:18:19.205000Z
4,34692381_2023-04-15T13:17:55.539433Z_WS.POZA.S...,2023-04-15T13:18:17.646000Z,2023-04-15T13:18:19.199000Z
...,...,...,...
323,35419441_2023-06-30T04:10:40.107601Z_WS.POZA.S...,2023-06-30T04:11:06.886000Z,2023-06-30T04:11:12.772000Z
324,35419441_2023-06-30T04:10:40.108841Z_WS.POZA.S...,2023-06-30T04:11:07.000000Z,2023-06-30T04:11:12.800000Z
325,35419441_2023-06-30T04:10:40.109398Z_WS.POZA.S...,2023-06-30T04:11:06.981000Z,2023-06-30T04:11:12.762000Z
326,35419441_2023-06-30T04:10:40.107451Z_WS.POZA.S...,2023-06-30T04:11:06.960000Z,2023-06-30T04:11:12.762000Z


In [3]:
#remove unwanted char
def sanitize_filename(name):
    return name.replace(".", "").replace(":", "").replace("-", "").replace("_", "")


In [5]:
mseed_dir = "miniSEED_files"
output_dir = "output_plots"
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Loop through each miniSEED file in the directory
for file_path in Path(mseed_dir).glob("*.MSEED"):
    mseed_name = sanitize_filename(file_path.name.strip())  # Sanitize the miniSEED file name

    # Check if the miniSEED file matches an entry in the CSV
    matched_row = None
    for _, row in pick_data.iterrows():
        if sanitize_filename(row['trace_name_original_Z']) == mseed_name:
            matched_row = row
            break

    if matched_row is None:
        print(f"No match found for file: {file_path.name}")
        continue

    print(f"Match found: {file_path.name}")

    # Extract P and S pick times
    try:
        p_pick = UTCDateTime(matched_row['trace_p_pick_time'])
        s_pick = UTCDateTime(matched_row['trace_s_pick_time'])
    except Exception as e:
        print(f"Error parsing pick times for {file_path.name}: {e}")
        continue

    # Read the miniSEED file
    stream = read(file_path)
    trace = stream[0]
    #taking the first trace only from the seed files 

# Apply a bandpass filter (e.g., 0.5–5 Hz)
    lowcut = 0.1 # Lower frequency (Hz)
    highcut = 15.0  # Upper frequency (Hz)
    trace_filtered = trace.copy()
    trace_filtered.filter("bandpass", freqmin=lowcut, freqmax=highcut)

    # Plot and save each trace in the stream
    for trace in stream:
        fig, ax = plt.subplots(figsize=(10, 6))

        # Plot the waveform
        times = trace.times("matplotlib")
        ax.plot(times, trace.data, label=f"Waveform: {trace.id}", color="#008080")

        # Add vertical lines for P and S picks
        ax.axvline(p_pick.matplotlib_date, color="red", linestyle="--", label="P-wave Pick")
        ax.axvline(s_pick.matplotlib_date, color="black", linestyle="--", label="S-wave Pick")

        # Customize the plot
        
        ax.set_title(f"Waveform and Picks: {trace.id}")
        ax.set_xlabel("Time (UTC)")
        ax.set_ylabel("Amplitude")
        ax.legend()
        ax.grid()

        # Format the x-axis for date/time
        fig.autofmt_xdate()

        # Save the plot
        output_file = Path(output_dir) / f"{file_path.stem}_{trace.id}.png"
        plt.savefig(output_file)
        plt.close()
        print(f"Plot saved to: {output_file}")

No match found for file: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DNZ.MSEED
Plot saved to: output_plots\34161341_2023-02-21T00.07.00.489723Z_WS.POZA.S5.DNZ_WS.POZA.S5.DNZ.png
No match found for file: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DNZ.MSEED
Plot saved to: output_plots\34161341_2023-02-21T00.07.00.490024Z_WS.POZA.S3.DNZ_WS.POZA.S3.DNZ.png
No match found for file: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN1.MSEED
No match found for file: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DN2.MSEED
Match found: 34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DNZ.MSEED
Plot saved to: output_plots\34161341_2023-02-21T00.07.00.490032Z_WS.POZA.S2.DNZ_WS.POZA.S2.